In [2]:
# check version of gluonts
# import gluonts
# gluonts.__version__

In [1]:
# !pip install gluonts=="0.4.1"
# change to 0.4.0 if any problems occur
import gluonts
gluonts.__version__

'0.4.1'

In [2]:
# Imports
import pprint
from functools import partial

import pandas as pd

from gluonts.dataset.repository.datasets import get_dataset
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions

from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deepstate import DeepStateEstimator

from gluonts.trainer import Trainer

import mxnet as mx
import numpy as np

INFO:root:Using CPU
INFO:root:Using CPU


In [4]:
datasets = [
#     "m4_hourly",
#     "m4_daily",
    "m4_weekly",
#     "m4_monthly",
#     "m4_quarterly",
#     "m4_yearly",
]

epochs = 1
num_batches_per_epoch = 5
seed = 42

estimators = [
    partial(
        DeepAREstimator,
        trainer=Trainer(
            epochs=epochs, 
            num_batches_per_epoch=num_batches_per_epoch,
            ctx="cpu"
        ),
    ),
    partial(
        DeepStateEstimator,
        trainer=Trainer(
            epochs=epochs, 
            num_batches_per_epoch=num_batches_per_epoch,
            ctx='cpu'
        ),
    ),
#     partial(
#         MQCNNEstimator,
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
]


def evaluate(dataset_name, estimator):
    dataset = get_dataset(dataset_name)
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,
        cardinality=[
            feat_static_cat.cardinality
            for feat_static_cat in dataset.metadata.feat_static_cat
        ],
    )

    print(f"evaluating {estimator} on {dataset}")

    predictor = estimator.train(training_data=dataset.train, 
                                validation_data=dataset.test)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    pprint.pprint(agg_metrics)

    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    return eval_dict


if __name__ == "__main__":
    
    mx.random.seed(seed)
    np.random.seed(seed)


    results = []
    for dataset_name in datasets:
        for estimator in estimators:
            # catch exceptions that are happening during training to avoid failing the whole evaluation
            try:
                results.append(evaluate(dataset_name, estimator))
            except Exception as e:
                print(str(e))

    df = pd.DataFrame(results)

    sub_df = df[
        [
            "dataset",
            "estimator",
            "RMSE",
            "mean_wQuantileLoss",
            "MASE",
            "sMAPE",
            "MSIS",
        ]
    ]

    print(sub_df.to_string())

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_weekly.


evaluating gluonts.model.deepar._estimator.DeepAREstimator(cardinality=[359], cell_type="lstm", context_length=None, distr_output=gluonts.distribution.student_t.StudentTOutput(), dropout_rate=0.1, embedding_dimension=None, freq="W", lags_seq=None, num_cells=40, num_layers=2, num_parallel_samples=100, prediction_length=13, scaling=True, time_features=None, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=mxnet.context.Context("cpu", 0), epochs=1, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=5, patience=10, weight_decay=1e-08), use_feat_dynamic_real=False, use_feat_static_cat=True, use_feat_static_real=False) on TrainDatasets(metadata=<MetaData freq='W' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='359'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=13>, train=<gluonts.dataset.common.FileDataset 

INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 5/5 [00:03<00:00,  1.40it/s, avg_epoch_loss=8.89]
INFO:root:Epoch[0] Elapsed time 3.683 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=8.891572
0it [00:00, ?it/s]INFO:root:Number of parameters in DeepARTrainingNetwork: 49593
11it [00:06,  1.72it/s, validation_avg_epoch_loss=8.6]
INFO:root:Epoch[0] Elapsed time 6.422 seconds
INFO:root:Epoch[0] Evaluation metric 'validation_epoch_loss'=8.598819
INFO:root:Loading parameters from best epoch (0)
INFO:root:Final loss: 8.598819047976763 (occurred at epoch 0)
INFO:root:End model training
Running evaluation: 100%|██████████| 359/359 [00:50<00:00,  7.08it/s]


{'Coverage[0.1]': 0.0,
 'Coverage[0.2]': 0.0006428112277694451,
 'Coverage[0.3]': 0.057424469680737035,
 'Coverage[0.4]': 0.3634026140989921,
 'Coverage[0.5]': 0.6989500749946436,
 'Coverage[0.6]': 0.8322262695521745,
 'Coverage[0.7]': 0.9286479537175917,
 'Coverage[0.8]': 0.9712877651596312,
 'Coverage[0.9]': 0.9869295050353545,
 'MAE_Coverage': 0.1662857414946552,
 'MASE': 18.100461257103195,
 'MSE': 3339515.580839159,
 'MSIS': 217.2803119465804,
 'ND': 0.18986661310966307,
 'NRMSE': 0.3329301626481846,
 'QuantileLoss[0.1]': 4969770.461747741,
 'QuantileLoss[0.2]': 5451670.840563965,
 'QuantileLoss[0.3]': 4326091.933265686,
 'QuantileLoss[0.4]': 3422400.47399292,
 'QuantileLoss[0.5]': 4863792.010505676,
 'QuantileLoss[0.6]': 6500976.791189576,
 'QuantileLoss[0.7]': 7642628.755805971,
 'QuantileLoss[0.8]': 7688629.8313385,
 'QuantileLoss[0.9]': 6106946.0082534775,
 'RMSE': 1827.4341522580667,
 'abs_error': 4863792.048233032,
 'abs_target_mean': 5488.941397566194,
 'abs_target_sum': 25

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_weekly.


evaluating gluonts.model.deepstate._estimator.DeepStateEstimator(add_trend=False, cardinality=[359], cell_type="lstm", dropout_rate=0.1, embedding_dimension=None, freq="W", innovation_bounds=gluonts.distribution.lds.ParameterBounds(lower=1e-06, upper=0.01), issm=None, noise_std_bounds=gluonts.distribution.lds.ParameterBounds(lower=1e-06, upper=1.0), num_cells=40, num_layers=2, num_parallel_samples=100, num_periods_to_train=4, past_length=None, prediction_length=13, prior_cov_bounds=gluonts.distribution.lds.ParameterBounds(lower=1e-06, upper=1.0), scaling=True, time_features=None, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=mxnet.context.Context("cpu", 0), epochs=1, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=5, patience=10, weight_decay=1e-08), use_feat_dynamic_real=False, use_feat_static_cat=True) on TrainDatasets(metadata=<MetaData freq='W' target=None feat_stat

INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 5/5 [01:28<00:00, 17.63s/it, avg_epoch_loss=0.486]
INFO:root:Epoch[0] Elapsed time 88.162 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=0.486151
0it [00:00, ?it/s]INFO:root:Number of parameters in DeepStateTrainingNetwork: 50821
12it [02:50, 14.18s/it, validation_avg_epoch_loss=0.416]
INFO:root:Epoch[0] Elapsed time 170.454 seconds
INFO:root:Epoch[0] Evaluation metric 'validation_epoch_loss'=0.415844
INFO:root:Loading parameters from best epoch (0)
INFO:root:Final loss: 0.415844344827126 (occurred at epoch 0)
INFO:root:End model training
Running evaluation: 100%|██████████| 359/359 [03:27<00:00,  1.73it/s] 


{'Coverage[0.1]': 0.018427255196057427,
 'Coverage[0.2]': 0.05785301049925003,
 'Coverage[0.3]': 0.14913220484251138,
 'Coverage[0.4]': 0.28069423612599104,
 'Coverage[0.5]': 0.47396614527533765,
 'Coverage[0.6]': 0.7008785086779517,
 'Coverage[0.7]': 0.8782944075423179,
 'Coverage[0.8]': 0.9335761731304905,
 'Coverage[0.9]': 0.9558602956931649,
 'MAE_Coverage': 0.10983739256719749,
 'MASE': 12.03790902786159,
 'MSE': 2377691.19072159,
 'MSIS': 128.18162978678365,
 'ND': 0.15186060234158183,
 'NRMSE': 0.28092418545177844,
 'QuantileLoss[0.1]': 2716411.7707992555,
 'QuantileLoss[0.2]': 3856060.539926148,
 'QuantileLoss[0.3]': 4243706.650212097,
 'QuantileLoss[0.4]': 4176491.4996490483,
 'QuantileLoss[0.5]': 3890196.273864746,
 'QuantileLoss[0.6]': 3754677.283862305,
 'QuantileLoss[0.7]': 3862798.9599197386,
 'QuantileLoss[0.8]': 3723629.969848633,
 'QuantileLoss[0.9]': 2742259.542593383,
 'RMSE': 1541.9763911038294,
 'abs_error': 3890196.269958496,
 'abs_target_mean': 5488.941397566194,

In [5]:
df[["dataset", "estimator", "MASE", "sMAPE", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]]

,dataset,estimator,MASE,sMAPE,wQuantileLoss[0.5],wQuantileLoss[0.9]
0,m4_weekly,DeepAREstimator,18.100461,0.199864,0.189867,0.238395
1,m4_weekly,DeepStateEstimator,12.037909,0.167949,0.151861,0.107049


In [6]:
print("Done!")

Done!
